In [3]:
# CELL 1 — Phone (IP camera base URL) capture calibration frames + calibrate camera (8x8 squares => 7x7 inner corners)
# Uses CAM_BASE (e.g., http://192.168.1.70:8080/) and auto-finds the actual stream endpoint.
# Captures into calib_frames_phone/ and saves intrinsics to camera_intrinsics_phone.npz

import cv2
import numpy as np
import glob
import re
import urllib.request
import urllib.parse
from pathlib import Path

# ============================
# CONFIG (phone + board)
# ============================
PRE_CALIBRATED = True # True = load intrinsics and skip capture+calibration

CAM_BASE = "http://192.168.1.70:8080/"  # base control page that works in your browser

CALIB_DIR = Path("calib_frames_phone")
INTRINSICS_PATH = "camera_intrinsics_phone.npz"

BOARD_OUTER_MM = 336.0
SQUARE_MM = BOARD_OUTER_MM / 8.0
SQUARE_M = SQUARE_MM / 1000.0
PATTERN_SIZE = (7, 7)  # inner corners for 8x8 squares

# Capture preview sizing (saved images remain full stream resolution)
PREVIEW_MAX_W = 1600
PREVIEW_MAX_H = 900

# ============================
# URL discovery helpers
# ============================
COMMON_STREAM_PATHS = [
    "video",
    "videofeed",
    "mjpeg", "mjpegfeed",
    "live", "stream",
    "?action=stream",
    "video?x.mjpeg",
    "mjpg/video.mjpg",
]

def _fetch_html(url, timeout=3.0):
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        return r.read().decode("utf-8", errors="ignore")

def _extract_stream_urls_from_html(base_url, html):
    candidates = set()
    for m in re.finditer(r'(?:href|src)\s*=\s*["\']([^"\']+)["\']', html, flags=re.IGNORECASE):
        u = m.group(1).strip()
        if not u:
            continue
        low = u.lower()
        if any(k in low for k in ["mjpeg", "mjpg", "videofeed", "video", "stream", "action=stream"]):
            candidates.add(urllib.parse.urljoin(base_url, u))
    return list(candidates)

def _try_open(url):
    backends = [None, cv2.CAP_MSMF]
    try:
        backends.append(cv2.CAP_FFMPEG)
    except Exception:
        pass

    for be in backends:
        try:
            cap = cv2.VideoCapture(url) if be is None else cv2.VideoCapture(url, be)
        except Exception:
            continue
        if cap is None or not cap.isOpened():
            try:
                cap.release()
            except:
                pass
            continue
        ok, frame = cap.read()
        if ok and frame is not None and frame.size > 0:
            return cap, frame
        cap.release()
    return None, None

def open_cam_url():
    base = CAM_BASE if CAM_BASE.endswith("/") else (CAM_BASE + "/")

    html = _fetch_html(base, timeout=3.0)
    scraped = _extract_stream_urls_from_html(base, html)
    fallbacks = [urllib.parse.urljoin(base, p) for p in COMMON_STREAM_PATHS]

    seen = set()
    candidates = []
    for u in scraped + fallbacks:
        if u not in seen:
            seen.add(u)
            candidates.append(u)

    for u in candidates:
        cap, frame = _try_open(u)
        if cap is not None:
            return cap, u, frame

    raise RuntimeError(
        "Browser opens the base page, but OpenCV can't find a readable stream endpoint.\n"
        "Fix:\n"
        "  - Open the base page in your browser, right-click the video -> Copy video address,\n"
        "    and set CAM_BASE to that direct stream URL (or paste it to me).\n"
    )

# ============================
# Calibration helpers
# ============================
def mk_object_points(pattern_size, square_m):
    nx, ny = pattern_size
    objp = np.zeros((nx * ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) * square_m
    return objp

print(f"Chessboard: 8x8 squares, outer={BOARD_OUTER_MM:.1f} mm => square={SQUARE_MM:.3f} mm ({SQUARE_M:.6f} m)")
print(f"Inner corners: {PATTERN_SIZE[0]}x{PATTERN_SIZE[1]}")
print("Calib folder:", str(CALIB_DIR))
print("Intrinsics file:", INTRINSICS_PATH)

# ============================
# (A) Capture frames (optional)
# ============================
if not PRE_CALIBRATED:
    CALIB_DIR.mkdir(parents=True, exist_ok=True)

    cap, chosen_url, first = open_cam_url()
    h0, w0 = first.shape[:2]
    print("Opened stream:", chosen_url)
    print(f"Stream resolution: {w0}x{h0}")
    print("SPACE=save | ESC=quit")

    idx = 0
    while True:
        ok, frame = cap.read()
        if not ok or frame is None:
            print("Frame read failed; exiting.")
            break

        preview = cv2.flip(frame, 1)

        ph, pw = preview.shape[:2]
        scale = min(PREVIEW_MAX_W / pw, PREVIEW_MAX_H / ph, 1.0)
        if scale < 1.0:
            preview = cv2.resize(preview, (int(pw * scale), int(ph * scale)), interpolation=cv2.INTER_AREA)

        cv2.imshow("PHONE: SPACE=save | ESC=quit", preview)
        key = cv2.waitKey(1) & 0xFF

        if key == 27:
            print("ESC pressed; exiting.")
            break

        if key == 32:
            fname = CALIB_DIR / f"frame_{idx:04d}.jpg"
            ok_write = cv2.imwrite(str(fname), frame, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
            print(f"Saved {fname}" if ok_write else f"Failed to save {fname}")
            idx += 1

    cap.release()
    cv2.destroyAllWindows()
else:
    print("Skipping capture (PRE_CALIBRATED=True)")

# ============================
# (B) Calibrate or load intrinsics
# ============================
if PRE_CALIBRATED:
    z = np.load(INTRINSICS_PATH, allow_pickle=True)
    K = z["camera_matrix"]
    dist = z["dist_coeffs"]
    image_size = tuple(z["image_size"])
    print("Loaded intrinsics from", INTRINSICS_PATH)
    print("image_size:", image_size)

else:
    objp = mk_object_points(PATTERN_SIZE, SQUARE_M)

    objpoints = []
    imgpoints = []
    image_size = None

    SB_FLAGS = (cv2.CALIB_CB_EXHAUSTIVE | cv2.CALIB_CB_ACCURACY)
    CLASSIC_FLAGS = (cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE)
    term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))

    img_files = sorted(glob.glob(str(CALIB_DIR / "*.jpg")))
    if not img_files:
        raise RuntimeError(f"No images found in {CALIB_DIR}. Capture frames first (PRE_CALIBRATED=False).")

    for fname in img_files:
        img = cv2.imread(fname)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if image_size is None:
            image_size = (gray.shape[1], gray.shape[0])

        g = clahe.apply(gray)

        ok, corners = cv2.findChessboardCornersSB(g, PATTERN_SIZE, SB_FLAGS)
        if not ok:
            ok, corners = cv2.findChessboardCornersSB(255 - g, PATTERN_SIZE, SB_FLAGS)

        if not ok:
            ok, corners = cv2.findChessboardCorners(g, PATTERN_SIZE, CLASSIC_FLAGS)
            if ok:
                corners = cv2.cornerSubPix(g, corners, (11, 11), (-1, -1), term)

        if not ok:
            continue

        objpoints.append(objp.copy())
        imgpoints.append(corners)

    n = len(imgpoints)
    print("chessboard detections:", n, "image_size:", image_size)
    if n == 0:
        raise RuntimeError("No usable chessboard detections. Improve lighting/contrast, reduce glare, fill frame more, add varied angles.")

    rms, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_size, None, None)

    np.savez(
        INTRINSICS_PATH,
        rms=float(rms),
        camera_matrix=K,
        dist_coeffs=dist,
        image_size=np.array(image_size, dtype=int),
        pattern_size=np.array(PATTERN_SIZE, dtype=int),
        square_length_m=float(SQUARE_M),
        board_outer_mm=float(BOARD_OUTER_MM),
        cam_base=str(CAM_BASE),
    )

    print("Saved intrinsics to", INTRINSICS_PATH)
    print("RMS reprojection error:", float(rms))


Chessboard: 8x8 squares, outer=336.0 mm => square=42.000 mm (0.042000 m)
Inner corners: 7x7
Calib folder: calib_frames_phone
Intrinsics file: camera_intrinsics_phone.npz
Skipping capture (PRE_CALIBRATED=True)
Loaded intrinsics from camera_intrinsics_phone.npz
image_size: (np.int64(1920), np.int64(1080))


In [4]:
import cv2
import numpy as np
import re
import urllib.request
import urllib.parse

# Assumes: K, dist already exist (and are for the SAME camera you're about to open)

# ----------------------------
# Choose ONE source
# ----------------------------
SOURCE = "phone_url"      # "device" or "phone_url"

# device webcam (if SOURCE="device")
CAM_INDEX = 0
CAPTURE_W = 3840
CAPTURE_H = 2160

# phone base page (if SOURCE="phone_url")
CAM_BASE = "http://192.168.1.70:8080/"

# ----------------------------
# Display layout
# ----------------------------
TILE_W = 640
TILE_H = 360

DIFF_THRESH = 25
BLUR_K = 5
ALPHA_NEWK = 0.0  # 0=crop to valid pixels (reduces black borders), 1=keep all pixels

# ----------------------------
# Open phone stream by discovering the actual MJPEG endpoint
# ----------------------------
COMMON_STREAM_PATHS = [
    "video",
    "videofeed",
    "mjpeg", "mjpegfeed",
    "live", "stream",
    "?action=stream",
    "video?x.mjpeg",
    "mjpg/video.mjpg",
]

def _fetch_html(url, timeout=3.0):
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        return r.read().decode("utf-8", errors="ignore")

def _extract_stream_urls_from_html(base_url, html):
    candidates = set()
    for m in re.finditer(r'(?:href|src)\s*=\s*["\']([^"\']+)["\']', html, flags=re.IGNORECASE):
        u = m.group(1).strip()
        if not u:
            continue
        low = u.lower()
        if any(k in low for k in ["mjpeg", "mjpg", "videofeed", "video", "stream", "action=stream"]):
            candidates.add(urllib.parse.urljoin(base_url, u))
    return list(candidates)

def _try_open(url):
    # try default + MSMF; CAP_FFMPEG may not exist in your build
    backends = [None, cv2.CAP_MSMF]
    try:
        backends.append(cv2.CAP_FFMPEG)
    except Exception:
        pass

    for be in backends:
        cap = cv2.VideoCapture(url) if be is None else cv2.VideoCapture(url, be)
        if not cap.isOpened():
            cap.release()
            continue
        ok, frame = cap.read()
        if ok and frame is not None and frame.size > 0:
            return cap, frame
        cap.release()
    return None, None

def open_phone_cap():
    base = CAM_BASE if CAM_BASE.endswith("/") else (CAM_BASE + "/")
    html = _fetch_html(base, timeout=3.0)
    scraped = _extract_stream_urls_from_html(base, html)
    fallbacks = [urllib.parse.urljoin(base, p) for p in COMMON_STREAM_PATHS]

    seen = set()
    candidates = []
    for u in scraped + fallbacks:
        if u not in seen:
            seen.add(u)
            candidates.append(u)

    for u in candidates:
        cap, frame = _try_open(u)
        if cap is not None:
            return cap, u, frame

    raise RuntimeError(
        "Failed to open phone stream.\n"
        "Open CAM_BASE in your browser, right-click the live video, copy the *video/stream* URL, and use that directly."
    )

def open_device_cap():
    cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAPTURE_W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAPTURE_H)
    if not cap.isOpened():
        cap.release()
        raise RuntimeError("Failed to open camera device")
    ok, frame = cap.read()
    if not ok or frame is None:
        cap.release()
        raise RuntimeError("Opened device camera but failed to read a frame")
    return cap, f"device:{CAM_INDEX}", frame

# ----------------------------
# Tiles
# ----------------------------
def resize_tile(img):
    return cv2.resize(img, (TILE_W, TILE_H), interpolation=cv2.INTER_AREA)

def label(img, text):
    out = img.copy()
    cv2.putText(out, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 4, cv2.LINE_AA)
    cv2.putText(out, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2, cv2.LINE_AA)
    return out

# ----------------------------
# Run sanity check (always)
# ----------------------------
if SOURCE == "phone_url":
    cap, src, first = open_phone_cap()
elif SOURCE == "device":
    cap, src, first = open_device_cap()
else:
    raise ValueError('SOURCE must be "phone_url" or "device"')

print("ESC=quit | source:", src)

newK = None
last_size = None

while True:
    ok, frame = cap.read()
    if not ok or frame is None:
        break

    h, w = frame.shape[:2]
    if newK is None or last_size != (w, h):
        newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), ALPHA_NEWK)
        last_size = (w, h)

    und = cv2.undistort(frame, K, dist, None, newK)

    g_raw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    g_und = cv2.cvtColor(und, cv2.COLOR_BGR2GRAY)

    diff = cv2.absdiff(g_raw, g_und)
    if BLUR_K and BLUR_K > 0:
        k = BLUR_K if (BLUR_K % 2 == 1) else (BLUR_K + 1)
        diff = cv2.GaussianBlur(diff, (k, k), 0)

    diff_color = cv2.applyColorMap(diff, cv2.COLORMAP_TURBO)

    mask = (diff >= DIFF_THRESH).astype(np.uint8) * 255
    k5 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k5)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, k5)

    overlay = und.copy()
    m = (mask > 0)

    if m.any():
        over_f = overlay.astype(np.float32)
        red = np.array([0, 0, 255], dtype=np.float32)
        over_f[m] = 0.6 * over_f[m] + 0.4 * red
        overlay = over_f.astype(np.uint8)

    hotspot_pct = 100.0 * float(np.count_nonzero(mask)) / float(mask.size)

    overlay = label(overlay, f"hotspots: {hotspot_pct:.2f}%")

    raw_t  = label(resize_tile(frame),     "RAW")
    und_t  = label(resize_tile(und),       "UNDISTORTED")
    diff_t = label(resize_tile(diff_color),"ABS DIFF")
    over_t = label(resize_tile(overlay),   "HOTSPOTS")

    top = np.hstack([raw_t, und_t])
    bot = np.hstack([diff_t, over_t])
    mosaic = np.vstack([top, bot])

    cv2.imshow("Calibration Sanity Check", mosaic)

    if (cv2.waitKey(1) & 0xFF) == 27:
        break

cap.release()
cv2.destroyAllWindows()


ESC=quit | source: http://192.168.1.70:8080/video


KeyboardInterrupt: 

In [ ]:
# CELL 2 — Live measurement (AUTO camera select: phone stream if available, else local webcam)
# Standalone: loads intrinsics from INTRINSICS_PATH, opens the SAME camera source automatically.
#  - If phone stream opens: uses it
#  - Else: falls back to local webcam CAM_INDEX
#
# Keys:
#   LClick add/drag | LClick after 4 pts cycles points | RClick toggles zoom | C clear | P print | ESC quit

import cv2
import numpy as np
import re
import urllib.request
import urllib.parse

# ----------------------------
# CONFIG (your chessboard)
# ----------------------------
PATTERN_SIZE = (7, 7)                 # 8x8 squares => 7x7 inner corners
SQUARE_M = (336.0 / 8.0) / 1000.0     # 336mm outer => 42mm squares => 0.042m

# Intrinsics file to use (should match the camera you intend to use)
INTRINSICS_PATH = "camera_intrinsics_phone.npz"  # change if you saved a different file

# Phone base page (AUTO tries to find actual stream endpoint)
CAM_BASE = "http://192.168.1.70:8080/"

# Fallback local webcam
CAM_INDEX = 0
CAPTURE_W = 3840
CAPTURE_H = 2160

# ----------------------------
# Performance knobs
# ----------------------------
DETECT_EVERY_N = 6
DETECT_SCALE = 0.45
CLAHE_ON = True
SB_FIRST = True

# ----------------------------
# Display
# ----------------------------
DISPLAY_MAX_W = 1600
DISPLAY_MAX_H = 900
disp_scale = 1.0

# ----------------------------
# Point rendering (ABSOLUTE screen pixels, same in main + zoom)
# ----------------------------
POINT_RING_R = 8
POINT_RING_TH = 2
POINT_CENTER_R = 1
POINT_HIT_R_FULL = 12
LABEL_SCALE = 0.6

# ----------------------------
# Zoom
# ----------------------------
zoom_on = False
zoom_center = None
zoom_factor = 4
zoom_radius = 180
zoom_win = "zoom"

# ----------------------------
# State
# ----------------------------
pts = []
drag_i = None
cycle_i = 0
last_dims_m = None

newK = None
last_size = None

_last_zoom_origin = None
_cached_have_H = False
_cached_Hinv = None
_cached_corners_full = None
_frame_i = 0

# ----------------------------
# Load intrinsics (K, dist)
# ----------------------------
z = np.load(INTRINSICS_PATH, allow_pickle=True)
K = z["camera_matrix"]
dist = z["dist_coeffs"]

# ----------------------------
# Camera open (AUTO: phone stream first, else device)
# ----------------------------
COMMON_STREAM_PATHS = [
    "video",
    "videofeed",
    "mjpeg", "mjpegfeed",
    "live", "stream",
    "?action=stream",
    "video?x.mjpeg",
    "mjpg/video.mjpg",
]

def _fetch_html(url, timeout=3.0):
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        return r.read().decode("utf-8", errors="ignore")

def _extract_stream_urls_from_html(base_url, html):
    candidates = set()
    for m in re.finditer(r'(?:href|src)\s*=\s*["\']([^"\']+)["\']', html, flags=re.IGNORECASE):
        u = m.group(1).strip()
        if not u:
            continue
        low = u.lower()
        if any(k in low for k in ["mjpeg", "mjpg", "videofeed", "video", "stream", "action=stream"]):
            candidates.add(urllib.parse.urljoin(base_url, u))
    return list(candidates)

def _try_open_url(url):
    backends = [None, cv2.CAP_MSMF]
    try:
        backends.append(cv2.CAP_FFMPEG)
    except Exception:
        pass

    for be in backends:
        cap = cv2.VideoCapture(url) if be is None else cv2.VideoCapture(url, be)
        if not cap.isOpened():
            cap.release()
            continue
        ok, frame = cap.read()
        if ok and frame is not None and frame.size > 0:
            return cap, frame
        cap.release()
    return None, None

def _open_phone():
    base = CAM_BASE if CAM_BASE.endswith("/") else (CAM_BASE + "/")
    try:
        html = _fetch_html(base, timeout=3.0)
    except Exception:
        return None, None, None

    scraped = _extract_stream_urls_from_html(base, html)
    fallbacks = [urllib.parse.urljoin(base, p) for p in COMMON_STREAM_PATHS]

    seen = set()
    candidates = []
    for u in scraped + fallbacks:
        if u not in seen:
            seen.add(u)
            candidates.append(u)

    for u in candidates:
        cap, frame = _try_open_url(u)
        if cap is not None:
            return cap, u, frame

    return None, None, None

def _open_device():
    cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAPTURE_W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAPTURE_H)
    if not cap.isOpened():
        cap.release()
        return None, None
    ok, frame = cap.read()
    if not ok or frame is None or frame.size == 0:
        cap.release()
        return None, None
    return cap, frame

def open_cam_auto():
    cap, url, frame = _open_phone()
    if cap is not None:
        return cap, f"phone:{url}", frame
    cap2, frame2 = _open_device()
    if cap2 is not None:
        return cap2, f"device:{CAM_INDEX}", frame2
    raise RuntimeError("Couldn't open phone stream OR local webcam.")

# ----------------------------
# Geometry / drawing helpers
# ----------------------------
def compute_display_scale(h, w):
    return min(DISPLAY_MAX_W / w, DISPLAY_MAX_H / h, 1.0)

def to_full_res(x, y, scale):
    return int(x / scale), int(y / scale)

def to_disp(x_full, y_full, scale):
    return int(x_full * scale), int(y_full * scale)

def nearest_point_index_full(x, y, pts_list, r=POINT_HIT_R_FULL):
    if not pts_list:
        return None
    p = np.asarray(pts_list, dtype=np.float32)
    d2 = (p[:, 0] - x)**2 + (p[:, 1] - y)**2
    i = int(np.argmin(d2))
    return i if d2[i] <= r*r else None

def order_quad(pts4):
    pts4 = np.asarray(pts4, dtype=np.float32)
    c = pts4.mean(axis=0)
    ang = np.arctan2(pts4[:, 1] - c[1], pts4[:, 0] - c[0])
    pts4 = pts4[np.argsort(ang)]
    s = pts4.sum(axis=1)
    i0 = int(np.argmin(s))
    return np.roll(pts4, -i0, axis=0)

def quad_wh_m(world_xy4):
    p = np.asarray(world_xy4, dtype=np.float64)
    d01 = np.linalg.norm(p[1] - p[0])
    d12 = np.linalg.norm(p[2] - p[1])
    d23 = np.linalg.norm(p[3] - p[2])
    d30 = np.linalg.norm(p[0] - p[3])
    w = 0.5 * (d01 + d23)
    h = 0.5 * (d12 + d30)
    return w, h

def zoom_crop_bounds(img_shape, center_xy, radius):
    h, w = img_shape[:2]
    cx, cy = center_xy
    x1 = max(0, cx - radius); x2 = min(w, cx + radius)
    y1 = max(0, cy - radius); y2 = min(h, cy + radius)
    return x1, y1, x2, y2

def make_zoom_view(img_bgr, center_xy, radius, factor):
    x1, y1, x2, y2 = zoom_crop_bounds(img_bgr.shape, center_xy, radius)
    crop = img_bgr[y1:y2, x1:x2].copy()
    if crop.size == 0:
        return None, None
    zoom = cv2.resize(crop, None, fx=factor, fy=factor, interpolation=cv2.INTER_NEAREST)
    zh, zw = zoom.shape[:2]
    cv2.line(zoom, (zw//2, 0), (zw//2, zh), (0, 255, 0), 1)
    cv2.line(zoom, (0, zh//2), (zw, zh//2), (0, 255, 0), 1)
    return zoom, (x1, y1)

def draw_marker(img, x, y, color=(0, 255, 0), ring_r=POINT_RING_R, ring_th=POINT_RING_TH, center_r=POINT_CENTER_R):
    x = int(round(x)); y = int(round(y))
    cv2.circle(img, (x, y), ring_r, color, ring_th, lineType=cv2.LINE_AA)
    cv2.circle(img, (x, y), center_r, color, -1, lineType=cv2.LINE_AA)

def draw_text(img, lines, x=18, y=40, dy=34):
    for i, s in enumerate(lines):
        yy = y + i * dy
        cv2.putText(img, s, (x, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0,0,0), 5)
        cv2.putText(img, s, (x, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (255,255,255), 2)

# ----------------------------
# Mouse callbacks
# ----------------------------
main_win = "Measure: LClick add/drag | RClick zoom toggle | C clear | P print | ESC quit"

def mouse_zoom(event, x, y, flags, param):
    global pts, drag_i, cycle_i, zoom_on, zoom_center, _last_zoom_origin

    if not zoom_on or zoom_center is None or _last_zoom_origin is None:
        return

    ox, oy = _last_zoom_origin
    fx = float(ox + (x / zoom_factor))
    fy = float(oy + (y / zoom_factor))

    if event == cv2.EVENT_RBUTTONDOWN:
        zoom_on = False
        zoom_center = None
        try:
            cv2.destroyWindow(zoom_win)
        except:
            pass
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        i = nearest_point_index_full(fx, fy, pts, r=POINT_HIT_R_FULL)
        if i is not None:
            drag_i = i
            pts[drag_i] = (fx, fy)
            return

        if len(pts) == 4:
            drag_i = cycle_i
            cycle_i = (cycle_i + 1) % 4
            pts[drag_i] = (fx, fy)
            return

        if len(pts) < 4:
            pts.append((fx, fy))
            return

    elif event == cv2.EVENT_MOUSEMOVE:
        if drag_i is not None:
            pts[drag_i] = (fx, fy)

    elif event == cv2.EVENT_LBUTTONUP:
        drag_i = None

def mouse_main(event, x, y, flags, param):
    global pts, drag_i, cycle_i, zoom_on, zoom_center, disp_scale

    fx, fy = to_full_res(x, y, disp_scale)

    if event == cv2.EVENT_RBUTTONDOWN:
        if not zoom_on:
            zoom_center = (fx, fy)
            zoom_on = True
            cv2.namedWindow(zoom_win)
            cv2.setMouseCallback(zoom_win, mouse_zoom)
        else:
            zoom_on = False
            zoom_center = None
            try:
                cv2.destroyWindow(zoom_win)
            except:
                pass
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        i = nearest_point_index_full(fx, fy, pts, r=POINT_HIT_R_FULL)

        if i is not None:
            drag_i = i
            pts[drag_i] = (float(fx), float(fy))
            return

        if len(pts) == 4:
            drag_i = cycle_i
            cycle_i = (cycle_i + 1) % 4
            pts[drag_i] = (float(fx), float(fy))
            return

        if len(pts) < 4:
            pts.append((float(fx), float(fy)))
            return

    elif event == cv2.EVENT_MOUSEMOVE:
        if drag_i is not None:
            pts[drag_i] = (float(fx), float(fy))

    elif event == cv2.EVENT_LBUTTONUP:
        drag_i = None

# ----------------------------
# Chessboard detection precompute
# ----------------------------
nx, ny = PATTERN_SIZE
world_xy = (np.mgrid[0:nx, 0:ny].T.reshape(-1, 2).astype(np.float32) * float(SQUARE_M))

SB_FLAGS = (cv2.CALIB_CB_EXHAUSTIVE | cv2.CALIB_CB_ACCURACY)
CLASSIC_FLAGS = (cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE)
term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)) if CLAHE_ON else None

def detect_board_fast(gray_full):
    if DETECT_SCALE != 1.0:
        g = cv2.resize(gray_full, (0, 0), fx=DETECT_SCALE, fy=DETECT_SCALE, interpolation=cv2.INTER_AREA)
    else:
        g = gray_full

    g1 = clahe.apply(g) if clahe is not None else g

    if SB_FIRST:
        ok, corners = cv2.findChessboardCornersSB(g1, PATTERN_SIZE, SB_FLAGS)
        if ok:
            corners = corners.reshape(-1, 2) / DETECT_SCALE
            return corners.reshape(-1, 1, 2).astype(np.float32)

        ok, corners = cv2.findChessboardCornersSB(255 - g1, PATTERN_SIZE, SB_FLAGS)
        if ok:
            corners = corners.reshape(-1, 2) / DETECT_SCALE
            return corners.reshape(-1, 1, 2).astype(np.float32)

    ok, corners = cv2.findChessboardCorners(g1, PATTERN_SIZE, CLASSIC_FLAGS)
    if ok:
        corners = cv2.cornerSubPix(g1, corners, (11, 11), (-1, -1), term)
        corners = corners.reshape(-1, 2) / DETECT_SCALE
        return corners.reshape(-1, 1, 2).astype(np.float32)

    ok, corners = cv2.findChessboardCorners(255 - g1, PATTERN_SIZE, CLASSIC_FLAGS)
    if ok:
        corners = cv2.cornerSubPix(255 - g1, corners, (11, 11), (-1, -1), term)
        corners = corners.reshape(-1, 2) / DETECT_SCALE
        return corners.reshape(-1, 1, 2).astype(np.float32)

    return None

# ----------------------------
# Camera (AUTO)
# ----------------------------
cap, src, first = open_cam_auto()
print("ESC=quit | C=clear | P=print | RClick toggle zoom | source:", src)

cv2.namedWindow(main_win)
cv2.setMouseCallback(main_win, mouse_main)

while True:
    ok, frame = cap.read()
    if not ok or frame is None:
        break

    h, w = frame.shape[:2]
    if newK is None or last_size != (w, h):
        newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), 1)
        last_size = (w, h)

    und = cv2.undistort(frame, K, dist, None, newK)
    gray = cv2.cvtColor(und, cv2.COLOR_BGR2GRAY)

    if (_frame_i % DETECT_EVERY_N) == 0 or _cached_Hinv is None:
        corners_full = detect_board_fast(gray)
        _cached_corners_full = corners_full
        _cached_have_H = False
        _cached_Hinv = None

        if corners_full is not None:
            img_pts = corners_full.reshape(-1, 2).astype(np.float32)
            H_world_to_img, _ = cv2.findHomography(world_xy, img_pts, 0)
            if H_world_to_img is not None:
                _cached_Hinv = np.linalg.inv(H_world_to_img)
                _cached_have_H = True

    have_H = _cached_have_H
    Hinv = _cached_Hinv
    corners_full = _cached_corners_full

    disp_scale = compute_display_scale(und.shape[0], und.shape[1])
    disp = und if disp_scale >= 1.0 else cv2.resize(
        und, (int(und.shape[1]*disp_scale), int(und.shape[0]*disp_scale)), interpolation=cv2.INTER_AREA
    )

    if corners_full is not None:
        corners_disp = corners_full.copy()
        corners_disp[:, 0, 0] *= disp_scale
        corners_disp[:, 0, 1] *= disp_scale
        cv2.drawChessboardCorners(disp, PATTERN_SIZE, corners_disp, True)

    if len(pts) > 0:
        for i, (x, y) in enumerate(pts):
            dx, dy = to_disp(x, y, disp_scale)
            draw_marker(disp, dx, dy)
            cv2.putText(disp, str(i+1), (dx+POINT_RING_R+4, dy-POINT_RING_R-2),
                        cv2.FONT_HERSHEY_SIMPLEX, LABEL_SCALE, (0,255,0), 1, lineType=cv2.LINE_AA)

        if len(pts) == 4:
            quad = order_quad(pts)
            quad_disp = (quad * disp_scale).astype(np.float32)
            cv2.polylines(disp, [quad_disp.astype(np.int32).reshape(-1, 1, 2)], True,
                          (0, 255, 0), 1, lineType=cv2.LINE_AA)

            if have_H:
                pts_img = quad.astype(np.float32).reshape(-1, 1, 2)
                pts_world = cv2.perspectiveTransform(pts_img, Hinv).reshape(-1, 2)
                w_m, h_m = quad_wh_m(pts_world)
                last_dims_m = (float(w_m), float(h_m))

    status = [f"board: {'OK' if have_H else 'NOT FOUND'} | points: {len(pts)}/4 | zoom: {'ON' if zoom_on else 'OFF'}"]
    if last_dims_m is not None:
        status.append(f"W: {last_dims_m[0]*1000.0:.1f} mm   H: {last_dims_m[1]*1000.0:.1f} mm")
    else:
        status.append("W: --   H: --")

    draw_text(disp, status)
    cv2.imshow(main_win, disp)

    if zoom_on:
        if zoom_center is None:
            zoom_center = (w // 2, h // 2)

        zoom_img, origin = make_zoom_view(und, zoom_center, zoom_radius, zoom_factor)
        _last_zoom_origin = origin

        if zoom_img is not None and origin is not None:
            ox, oy = origin

            for i, (px, py) in enumerate(pts):
                zx = int((px - ox) * zoom_factor)
                zy = int((py - oy) * zoom_factor)
                if 0 <= zx < zoom_img.shape[1] and 0 <= zy < zoom_img.shape[0]:
                    draw_marker(zoom_img, zx, zy)
                    cv2.putText(zoom_img, str(i+1), (zx+POINT_RING_R+4, zy-POINT_RING_R-2),
                                cv2.FONT_HERSHEY_SIMPLEX, LABEL_SCALE, (0,255,0), 1, lineType=cv2.LINE_AA)

            cv2.imshow(zoom_win, zoom_img)

    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break
    if key in (ord('c'), ord('C')):
        pts = []
        drag_i = None
        cycle_i = 0
        last_dims_m = None
    if key in (ord('p'), ord('P')):
        if last_dims_m is None:
            print("No measurement yet (need 4 points + board found).")
        else:
            print(f"W: {last_dims_m[0]*1000.0:.1f} mm | H: {last_dims_m[1]*1000.0:.1f} mm")

    _frame_i += 1

cap.release()
cv2.destroyAllWindows()

if last_dims_m is None:
    raise RuntimeError("No measurement captured (need 4 clicked points + board found).")

print("Measured dimensions:")
print(f"  Width : {last_dims_m[0]*1000.0:.1f} mm")
print(f"  Height: {last_dims_m[1]*1000.0:.1f} mm")


ESC=quit | C=clear | P=print | RClick toggle zoom | source: phone:http://192.168.1.70:8080/video
W: 105.4 mm | H: 96.6 mm
